In [2]:
import random
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn

import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import ipywidgets as widgets

from torchvision.utils import save_image, make_grid

import math


In [3]:
# device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

In [4]:
class diffusive_flow(nn.Module):
    def __init__(self):
        super(diffusive_flow, self).__init__()
        # self.t_range = t_range
        # self.embedding = nn.Embedding(t_range, 1)
        
        self.layers = nn.Sequential(
            nn.Linear(3, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 5),
            nn.ReLU(),
            nn.Linear(5, 5),
            nn.ReLU(),
            nn.Linear(5, 2),
        )
    
    def forward(self, data, t):
        x = data[0]
        y = data[1]
        embedded_t = self.embedding(torch.tensor(t))

        x_y = torch.tensor([x, y])
        concatenated = torch.cat((x_y, embedded_t), dim=0)

        out = self.layers(concatenated)
        return out

model = diffusive_flow()

In [5]:
# diffuse 관련 변수 및 함수 정의

def make_alpha_bars(min_beta, max_beta, t_range, device=device):
    betas = torch.linspace(min_beta, max_beta, t_range).to(device)
    alphas = 1 - betas

    return torch.tensor([torch.prod(alphas[:(i+1)]) for i in range(len(alphas))]).to(device)

def diffuse_x0(x0, t, alpha_bars, eta=None, device=device):
    n, d = x0.shape
    a_bar = alpha_bars[t-1]

    if eta is None:
        eta = torch.randn(n, d).to(device)
    
    xT = a_bar.sqrt().view(-1,1) * x0 + (1 - a_bar).sqrt().view(-1,1) * eta

    return xT

In [6]:
# make vector field
def vector_field(x, y):
    u = x*0.05
    v = y*0.05
    vf = torch.cat((u, v), dim=1)
    return vf

In [7]:
# grid 생성
def make_grid(lim, n, device=device):
    x = np.linspace(-lim, lim, n)
    y = np.linspace(-lim, lim, n)
    X, Y = np.meshgrid(x, y)   # 20x20, 20x20
    X = X.reshape([-1,1])   # 20^2 x 1
    Y = Y.reshape([-1,1])   # 20^2 x 1

    grid_points = np.concatenate([X, Y], axis=1)
    return torch.tensor(grid_points, device=device, dtype=torch.float32)

grid_points = make_grid(5, 20)

In [8]:
def draw_data(data, title_str='Data', color='blue'):
    data = data.cpu()

    plt.figure(figsize=(8, 6))
    plt.scatter(data[:,0], data[:,1], label='data', c=color, alpha=1)
    plt.xlabel('X')
    plt.xlabel('Y')
    plt.set_title(title_str)
    plt.grid(True)
    plt.show()

In [9]:
def draw_vector_field(grid_points, vf, scale=5, color='blue', width=0.003, title='Vector Field'):
    X = grid_points.cpu().detach().numpy()[:, 0]
    Y = grid_points.cpu().detach().numpy()[:, 1]
    U = vf.cpu().detach().numpy()[:, 0]
    V = vf.cpu().detach().numpy()[:, 1]

    plt.figure(figsize=(8, 6))
    plt.quiver(X, Y, U, V, scale=scale, color=color, width=width)
    plt.xlabel('X')
    plt.xlabel('Y')
    plt.title(title)
    plt.grid(True)
    plt.show()

In [ ]:
t_range = 200
min_beta = 1e-3
max_beta = 1e-1
alpha_bars = make_alpha_bars(min_beta, max_beta, t_range)

def make_training_data(data, alpha_bars, t_range=200, device=device):
    x0 = data.to(device)
    for t in range(t_range):
        xT = diffuse_x0(x0, t, alpha_bars, eta=)